<a href="https://colab.research.google.com/github/monicamons12/30DayMapChallenge/blob/Monica/OSM_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal --quiet
!apt install python3-rtree --quiet
!pip install git+git://github.com/geopandas/geopandas.git --quiet
!pip install descartes --quiet
!pip install osmnx==1.1.1
!pip install folium

In [2]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import folium
import matplotlib.pyplot as plt
import seaborn as sb

## Extracting Street Map from OSM
It is possible to extract networks from OSM using OSMNX package. 
Various ways to extract - by specifying city, by the input of bbox, from point/address.

network_type can be 'drive' or 'walk' or 'bike'

save_graph_geopackage can also be used to save the network graph as geopackage instead of shapefile as mentioned in the following code

In [ ]:
place = "Chennai, India"
graph = ox.graph.graph_from_place(place, network_type='drive')
ox.io.save_graph_shapefile(graph)

In [ ]:
len(graph)
ox.stats.street_length_total(graph)

In [8]:
nodes, streets = ox.graph_to_gdfs(graph)

In [ ]:
nodes.head()

In [ ]:
streets.head()

In [11]:
street_types = pd.DataFrame(streets["highway"].apply(pd.Series)[0].value_counts().reset_index())
street_types.columns = ["type", "count"]

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
sb.stripplot(y="type", x="count", data=street_types, ax=ax)
plt.tight_layout()
plt.savefig("stripplot.png")
#sb.barplot(y="type", x="count", data=street_types, ax=ax)
#plt.tight_layout()
#plt.savefig("barchart.png")

In [67]:

style = {'color': '#F7DC6F', 'weight':'1'}
m = folium.Map([13.1031, 80.1794],
               zoom_start=15,
               tiles="CartoDb positron")
streetsJson = folium.GeoJson(streets, style_function=lambda x: style).add_to(m)
folium.GeoJsonTooltip(fields=['name'])
m.save("streets.html")
m

## Extract buildings from OSM

In [20]:
buildings = ox.geometries.geometries_from_place(place, {'building': True})

In [ ]:
buildings.head()

In [ ]:
buildings.shape

In [ ]:
cols = ['amenity','building', 'name', 'tourism']
buildings[cols].head()

In [ ]:
list(buildings.columns)

In [ ]:
buildings["amenity"].apply(pd.Series)[0].value_counts()


In [ ]:

style_buildings = {'color':'#6C3483 ', 'fillColor': '#6C3483 ', 'weight':'1', 'fillOpacity' : 1}

m = folium.Map([13.1031, 80.1794],
               zoom_start=15,
               tiles="cartoDb positron")

buildingJson = folium.GeoJson(buildings[:1000], style_function=lambda x: style_buildings).add_to(m)
folium.GeoJsonTooltip(fields=['name','amenity'], labels=False).add_to(buildingJson)
m.save("buildings.html")
m

## Hospitals in Chennai

In [35]:
hospitals = ox.geometries.geometries_from_place(place, {'amenity':'hospital'})

In [ ]:
hospitals.head()

In [ ]:
hospitals.shape

In [ ]:
hospitals.columns

In [ ]:
columns =['name','name:en', 'healthcare', 'healthcare:speciality', 'opening_hours', 'operator:type', 'addr:street', 'addr:city','addr:postcode', 'contact:phone']
hospitals[columns].head()

In [76]:
hospital_points = hospitals[hospitals.geom_type == "Point"]

In [ ]:
m = folium.Map([13.1031, 80.1794], zoom_start=10, tiles="CartoDb dark_matter")
locs = zip(hospital_points.geometry.y, hospital_points.geometry.x)
for location in locs:
    folium.CircleMarker(location=location, popup=folium.Popup(location) , color = "#F4F6F6",   radius=2).add_to(m)
m.save("hospitals.html")
m

##